<a href="https://colab.research.google.com/github/XiDerYu/face-recognition/blob/main/face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python cmake face_recognition

In [ ]:
!mkdir facedata
!wget https://i.imgur.com/I6FpRmB.jpg -O facedata/Elon_Musk.jpg
!wget https://i.imgur.com/oIvgQbg.jpg -O facedata/Tim_Cook.jpg
!wget https://i.imgur.com/ynPcb47.jpg -O facedata/gay.jpg

!mkdir testdata
!wget https://i.imgur.com/fqfI2pe.jpg -O testdata/Elon_Musk1.jpg
!wget https://i.imgur.com/adsjyRg.jpg -O testdata/Elon_Musk2.jpg
!wget https://i.imgur.com/BUHEiZj.jpg -O testdata/Elon_Musk3.jpg
!wget https://i.imgur.com/xEdiRiT.jpg -O testdata/Elon_Musk4.jpg
!wget https://i.imgur.com/NO5QQUg.jpg -O testdata/Tim_Cook1.jpg
!wget https://i.imgur.com/ScL5zKk.jpg -O testdata/Tim_Cook2.jpg
!wget https://i.imgur.com/Nw1YmP6.jpg -O testdata/Elon_and_Cook.jpg
!wget https://i.imgur.com/zCY9QaB.jpg -O testdata/gay1.jpg
!wget https://i.imgur.com/JvKCoT0.jpg -O testdata/gay2.jpg
!wget https://i.imgur.com/q0aAIiJ.jpg -O testdata/gay3.jpg
!wget https://i.imgur.com/goyvzbI.png -O testdata/me.jpg
!wget https://i.imgur.com/Q3OPd19.png -O testdata/oscar1.jpg

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from skimage import io
from google.colab.patches import cv2_imshow
known_face_list = [
    {
        'name': 'Elon Musk',
        'filename': 'facedata/Elon_Musk.jpg',
        'encode': None,
    },
    {
        'name': 'Tim cook',
        'filename': 'facedata/Tim_Cook.jpg',
        'encode': None,        
    },
    {
        'name': 'Gay',
        'filename': 'facedata/gay.jpg',
        'encode': None,        
    }
]

In [ ]:
for data in known_face_list:
    img = io.imread(data['filename'])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    data['encode'] = face_recognition.face_encodings(img)[0]
print(known_face_list)

In [ ]:
dirl = os.listdir('testdata')
test_fn_list = []
for filename in dirl:
  if '.jpg' in filename:
    test_fn_list.append('testdata/' + filename)
print(test_fn_list)
known_face_encodes = [data['encode'] for data in known_face_list]
tolerance = 0.6

['testdata/Elon_Musk2.jpg', 'testdata/me.jpg', 'testdata/Elon_Musk4.jpg', 'testdata/Elon_Musk1.jpg', 'testdata/oscar1.jpg', 'testdata/Tim_Cook1.jpg', 'testdata/gay3.jpg', 'testdata/Tim_Cook2.jpg', 'testdata/Elon_Musk3.jpg', 'testdata/Elon_and_Cook.jpg', 'testdata/gay2.jpg', 'testdata/gay1.jpg']


In [ ]:
RED_COLOR = (200, 58, 76)
WHITE_COLOR = (255, 255, 255)

def draw_locations(img, match_results):
    for match_result in match_results:
        y1, x2, y2, x1 = match_result['location']
        cv2.rectangle(img, (x1, y1), (x2, y2), RED_COLOR, 2)
        cv2.rectangle(img, (x1, y2 + 35), (x2, y2), RED_COLOR, cv2.FILLED)
        cv2.putText(img, match_result['name'], (x1 + 10, y2 + 25), cv2.FONT_HERSHEY_COMPLEX, 0.8, WHITE_COLOR, 2)

In [ ]:
for fn in test_fn_list:
    match_results = []
    img = io.imread(fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    cur_face_locs = face_recognition.face_locations(img)
    cur_face_encodes = face_recognition.face_encodings(img, cur_face_locs, model='large')
    
    for cur_face_encode, cur_face_loc in zip(cur_face_encodes, cur_face_locs):
      face_distance_list = face_recognition.face_distance(known_face_encodes, cur_face_encode)
        
      min_distance_index = np.argmin(face_distance_list)
      if face_distance_list[min_distance_index] < tolerance:
          result = known_face_list[min_distance_index]['name']
      else:
          result = 'unknow'
            
      print(f'辨識檔案: {fn}, 辨識結果: {result}')
      match_results.append({
          'name': result,
          'location': cur_face_loc,
      })
      draw_locations(img, match_results)
    cv2_imshow(img)